<a href="https://colab.research.google.com/github/MarielaNina/-Guide-to-Advanced-LLM-Techniques-Public/blob/main/M%C3%B3dulo_4_Fine_Tuning_Eficiente_Especializando_seu_Pr%C3%B3prio_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Módulo 4: Fine-Tuning Eficiente - Especializando seu Próprio LLM

# Introdução

Até agora, tratamos os LLMs como "caixas-pretas" que manipulamos através de entradas inteligentes (prompts) e combinações de saídas (ensembles). Neste último módulo, vamos abrir a caixa e modificar o próprio cérebro do modelo. O Fine-Tuning (ou ajuste fino) é o processo de continuar o treinamento de um modelo pré-treinado em um dataset específico de uma nova tarefa.

Enquanto a engenharia de prompt adapta a tarefa ao modelo, o fine-tuning adapta o modelo à tarefa. Isso resulta em um modelo especialista, que não apenas entende as nuances da sua tarefa específica, mas também pode ser menor, mais rápido e mais barato de executar em produção.

No entanto, o fine-tuning completo de um LLM com bilhões de parâmetros é computacionalmente proibitivo para a maioria. Por isso, este módulo foca em técnicas de Fine-Tuning Eficiente em Parâmetros (PEFT - Parameter-Efficient Fine-Tuning).

# 1. O que é Fine-Tuning?

Um LLM pré-treinado, como o Llama 3, aprendeu uma vasta gama de conhecimentos sobre linguagem, fatos e raciocínio a partir de trilhões de palavras da internet. O fine-tuning pega esse conhecimento geral e o especializa.

O processo envolve apresentar ao modelo um dataset de exemplos para uma tarefa específica (ex: pares de "notícia" e "sentimento") e ajustar seus pesos internos através de backpropagation para minimizar o erro nessas novas predições. O resultado é um novo conjunto de pesos, criando um modelo que é um especialista na sua tarefa.

**O principal desafio é o custo:** o fine-tuning completo de um modelo bilhões de parâmetros pode exigir GPUs de alta performance, algo inacessível para a maioria dos pesquisadores e desenvolvedores.

# 2. LoRA: Low-Rank Adaptation

Para resolver o problema do custo, surgiram as técnicas PEFT (Parameter-Efficient Fine-Tuning). A mais popular delas é a LoRA (Low-Rank Adaptation), proposta por Hu et al. (2021) [1].

A intuição por trás do LoRA é que a "mudança" que o fine-tuning provoca nos pesos de um modelo pré-treinado (W) tem uma "baixa dimensão intrínseca". Em vez de treinar a matriz de pesos W inteira, que é enorme, o LoRA congela W e treina apenas duas matrizes muito menores (A e B) que aproximam a matriz de atualização (ΔW). Durante a inferência, a atualização é simplesmente somada aos pesos originais.




![](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/blog/133_trl_peft/lora-animated.gif)

Figura 1: Representação visual do LoRA. Os pesos pré-treinados (W) são congelados. Apenas as matrizes de baixa ordem (A e B) são treinadas, reduzindo drasticamente o número de parâmetros ajustáveis.

Isso reduz o número de parâmetros treináveis drasticamente, tornando o fine-tuning muito mais rápido e com um consumo de memória menor.

# 3. Quantização de Modelos

A quantização ataca outro gargalo: o tamanho do modelo em memória. Por padrão, os pesos de um LLM são armazenados como números de ponto flutuante de alta precisão (ex: 32-bit). A quantização reduz essa precisão para 8-bit ou até 4-bit.

Isso leva a um ponto importante: um modelo maior com pesos quantizados frequentemente supera um modelo menor com pesos de alta precisão, com um custo de hardware similar. A pesquisa em quantização extrema, como a proposta pela arquitetura BitNet que explora modelos de 1-bit (Wang et al., 2024) [3], mostra que a escala do modelo (mais parâmetros) muitas vezes compensa a perda de precisão de cada peso individual, mantendo um desempenho robusto com uma fração do custo de memória.

# 4. QLoRA

QLoRA (Quantized Low-Rank Adaptation), de Dettmers et al. (2023) [2], é a combinação genial das duas técnicas anteriores e que democratizou o fine-tuning de LLMs.

O processo é o seguinte:
1. Carrega-se um LLM pré-treinado e o quantiza para uma precisão menor.
2. Os pesos originais do modelo são congelados.
3. Adiciona-se os adaptadores LoRA (A e B).
4. Treina-se apenas os adaptadores LoRA.

Essa técnica é o que vai nos permitir realizar o fine-tuning de um modelo como o Llama 3 em uma única GPU gratuita do Google Colab.

In [ ]:
!pip install -q transformers bitsandbytes accelerate peft trl datasets

## 4.1. Configurações do Colab

Mesmo com toda essa otimização de desempenho, não é possível executar no ambiente padrão do Colab. Siga os passos abaixo para se conectar a uma GPU gratuitamente:
1. Clique com o botão esquerdo na seta ao lado do botão conectar que está localizado no canto superior direito.
2. Clique em "Alterar o tipo de ambiente de execução".
3. Selecione GPU
4. Clique em Salvar


## 4.2. Hugging Face

Para conseguir acesso ao modelo utilizaremos a biblioteca Hugging Face. Para isso siga o seguinte passo a passo:
1. Crie sua conta no [Hugging Face](https://huggingface.co/)
2. Solicite o [acesso](https://huggingface.co/meta-llama/Llama-3.2-1B) ao modelo

In [ ]:
from huggingface_hub import notebook_login
print("Por favor, insira seu token de acesso do Hugging Face:")
notebook_login()

## 4.3. Carregando o Modelo e o Tokenizador

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Llama-3.2-1B"

# Configuração do BitsAndBytes para quantização em 4-bit (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
# Carregando o modelo principal com a configuração de quantização
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

In [ ]:
# Carregando o tokenizador correspondente ao modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token # O pad_token é necessário para o SFTTrainer. Definimos como o token de fim de sequência (eos_token).
tokenizer.padding_side = "right"

## 4.4. Preparação do Dataset

Esses modelos são treinados para tarefa de prever o próximo token. Dessa forma, precisamos converter nosso dataset em um texto que o modelo deve aprender a prever. Nosso objetivo vai ser criar um prompt no formato "instrução -> resposta JSON", para que o modelo aprenda as classificar as emoções e gerar saídas estruturadas.

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Curso LLMs/dataset.csv")
df

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate

template = """Classifique a notícia abaixo conforme a polaridade e emoção universal de ekman.
Texto:
f1: {f1}
f2: {f2}
f3: {f3}

Resultado:
{{
    'f1': {{
        'polaridade': '{f1_polaridade}',
        'emocao': '{f1_emocao}'
    }},
    'f2': {{
        'polaridade': '{f2_polaridade}',
        'emocao': '{f2_emocao}'
    }},
    'f3': {{
        'polaridade': '{f3_polaridade}',
        'emocao': '{f3_emocao}'
    }}
}}
"""
prompt = PromptTemplate.from_template(template=template)

In [ ]:
def create_json_prompt(row):
    return {'text': prompt.invoke({
        'f1': row['f1'],
        'f1_polaridade': row['sentence1_polarity'],
        'f1_emocao': row['sentence1_sentiment'],
        'f2': row['f2'],
        'f2_polaridade': row['sentence2_polarity'],
        'f2_emocao': row['sentence2_sentiment'],
        'f3': row['f3'],
        'f3_polaridade': row['sentence3_polarity'],
        'f3_emocao': row['sentence3_sentiment'],
    }).text}

In [ ]:
from datasets import Dataset

hf_dataset = Dataset.from_pandas(df)
formatted_dataset = hf_dataset.map(create_json_prompt)

## 4.5. Configuração de Treinamento

In [ ]:
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from transformers import TrainingArguments

lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

In [ ]:
training_args = TrainingArguments(
    output_dir="./llama3-finetuned-json",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    num_train_epochs=3,
    logging_steps=25,
    learning_rate=4e-4,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=lora_config,
    args=training_args,
)

## 4.6. Iniciar o Treinamento e Salvar

Aqui será nescessário criar uma conta no wandb, na própria execução veem isntruções de como criar a conta.

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./llama3-json-adapter")

# 5. Integração com Langchain

In [1]:
!pip install -q langchain langchain_core langchain_community langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from typing import Literal
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

TipoPolaridade = Literal['Positivo', 'Negativo', 'Neutro']
TipoEmocao = Literal['Felicidade', 'Tristeza', 'Raiva', 'Nojo', 'Medo', 'Surpresa', 'Desprezo', 'Neutro']

class AnaliseFrase(BaseModel):
    """Define a estrutura para a análise de uma única frase."""
    polaridade: TipoPolaridade = Field(description="A polaridade da frase (Positivo, Negativo ou Neutro)")
    emocao: TipoEmocao = Field(description="A emoção universal de Ekman detectada na frase")

class ClassificacaoNoticia(BaseModel):
    """Define a estrutura completa do JSON de saída para a notícia."""
    f1: AnaliseFrase = Field(description="Análise da primeira frase da notícia")
    f2: AnaliseFrase = Field(description="Análise da segunda frase da notícia")
    f3: AnaliseFrase = Field(description="Análise da terceira frase da notícia")

parser = PydanticOutputParser(pydantic_object=ClassificacaoNoticia)

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline

pipe = pipeline(
    task="text-generation",
    model=model,             # Usando o modelo fine-tuned já carregado
    tokenizer=tokenizer,     # Usando o tokenizador já carregado
    max_new_tokens=250,      # Espaço suficiente para o JSON.
    temperature=0.01,        # Temperatura baixa para saídas mais consistentes e determinísticas.
    return_full_text=False,  # Importante: retorna apenas o texto gerado.
)

llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
template = """Classifique a noticia abaixo conforme a polaridade e emoção universal.
Texto:
f1: {f1_text}
f2: {f2_text}
f3: {f3_text}

Resultado:
"""
prompt = PromptTemplate(template=template)

chain = prompt | llm | parser

In [ ]:
resposta = chain.invoke({
    'f1_text': df.iloc[0]['f1'],
    'f2_text': df.iloc[0]['f2'],
    'f3_text': df.iloc[0]['f3'],
})
print(resposta)

# 6. Atividade Prática Final: Destilação de Conhecimento

## 6.1. O que é Destilação?

A destilação de conhecimento é uma técnica de fine-tuning com um objetivo diferente. Em vez de apenas especializar um modelo, a ideia é usar um modelo grande e poderoso (o "professor") para treinar um modelo muito menor e mais eficiente (o "aluno").

O processo é o seguinte:
1. **Geração de Pseudo-Rótulos:** Usamos nosso melhor modelo "professor" (que pode ser um LLM de ponta via API, ou o melhor ensemble que criamos nos módulos anteriores) para classificar um grande volume de dados não rotulados. As saídas do professor são tratadas como se fossem rótulos verdadeiros.
2. **Treinamento do Aluno:** Usamos esse dataset recém-rotulado para fazer o fine-tuning de um modelo "aluno" menor.

O resultado é um modelo pequeno, rápido e barato de executar, que aprendeu a imitar o comportamento do professor para uma tarefa específica. Ele não terá o conhecimento geral do professor, mas será um especialista altamente eficiente na tarefa para a qual foi destilado.

## 6.2. Atividade

Criar um classificador de sentimento pequeno e eficiente usando a técnica de destilação de conhecimento.

Passos:
1. **Escolha seu "Professor":** Use a sua implementação de Chain-of-Thought com saída estruturada do Módulo 2. O objetivo é que a saída do professor contenha não apenas a classificação final, mas também o raciocínio passo a passo.
2. **Gere um Dataset:** Use o seu modelo professor para classificar o nosso dataset. A "resposta" no seu novo dataset de fine-tuning será um JSON contendo tanto o raciocinio quanto a classificacao.
3. **Escolha seu "Aluno":** Selecione um modelo pequeno para ser o aluno.
4. **Treine o Aluno para Pensar:** Adapte o código da seção 5 para fazer o fine-tuning do seu modelo aluno no dataset que você acabou de gerar. Ao fazer isso, você não está apenas ensinando o aluno a classificar, mas a imitar o processo de raciocínio do professor.
5. **Avalie:** Compare o desempenho do seu novo modelo aluno com o do professor e com as abordagens dos módulos anteriores. O aluno treinado com raciocínio consegue um desempenho melhor do que o treinado apenas com rótulos?

# Conclusão do Tutorial

Parabéns por chegar ao final! Começamos com prompts simples, avançamos para debates complexos entre agentes e, finalmente, modificamos os próprios pesos de um LLM. Você agora possui um kit de ferramentas robusto para abordar qualquer tarefa de classificação de texto com LLMs.

# Referências

[1] Hu, E. J., et al. (2021). LoRA: Low-Rank Adaptation of Large Language Models. arXiv:2106.09685

[2] Dettmers, T., et al. (2023). QLoRA: Efficient Finetuning of Quantized LLMs. arXiv:2305.14314

[3] Wang, S., et al. (2024). The Era of 1-bit LLMs: All Large Language Models are in 1.58 Bits. arXiv:2402.17764